In [1]:
import pandas as pd
import gensim
import numpy as np
from gensim.models import word2vec
import preprocessing
from gensim.models import Phrases

<h1>Import preprocessed data</h1>

Data from file created in get_stack_data 

In [2]:
data = pd.read_csv('DB/Preprocessed_data.csv')

data

,id,title,tags,score,answers_processed,questions_processed,processed_title,questions
0,742,Class views in Django,django|views|oop,0.006867,needed use class based views wanted able use f...,view points function problem want change bit f...,class views django,"view points to a function, which can be a pr..."
1,773,How do I use itertools.groupby()?,itertools,0.106274,neato trick groupby run length encoding one li...,able find understandable explanation actually ...,use itertools groupby,I haven't been able to find an understandable...
2,2933,Create a directly-executable cross-platform GU...,user-interface|deployment|tkinter|release-mana...,0.039969,pysimplegui wraps tkinter works python also ru...,python works multiple platforms used desktop w...,create directly executable cross platform gui ...,Python works on multiple platforms and can be...
3,5102,How do you set up Python scripts to work in Ap...,apache|apache2,0.002123,yes mod python pretty confusing set httpd conf...,tried follow couple googled tutorials setting ...,set python scripts work apache,I tried to follow a couple of googled up tuto...
4,8154,Pylons error - 'MySQL server has gone away',mysql|pylons,0.000345,think fixed turns simple config error ini file...,using pylons serve simple web application seem...,pylons error mysql server gone away,I'm using Pylons to serve a simple web appl...
...,...,...,...,...,...,...,...,...
227399,69060114,Pytorch geometric: how to explain the input in...,machine-learning|graph|pytorch|data-science,-0.000545,okay think got understanding output dimensions...,reading pytorch geometric documentation page c...,pytorch geometric explain input code snippet,I am reading PyTorch geometric documentation ...
227400,69060216,Squaring a number using return values,function|return|return-value,-0.000446,instructions tell write also function anything...,okay sure pretty simple assignment lot trouble...,squaring number using return values,"Okay, so I'm sure this is a pretty simple ass..."
227401,69060696,How do I extract two dates from a string in py...,regex|web-scraping|re|findall,-0.000446,try instead output,returns gives first date get second date,extract two dates string python,returns: This gives me the first date...
227402,69060948,Python Input and Output?,input|output,-0.000446,line got user input got convert input like see...,working learning python reason output get alwa...,python input output,I am working on learning python and I for som...


From the dataframe tokenize sentence from the titles, questions and anwers.
First create lists from column and than tokenize it and store that in documents.

In [3]:
titles = data.processed_title.tolist()
questions = data.questions_processed.tolist()
answers = data.answers_processed.tolist()

docs1 = [_text.split() for _text in np.array(titles)]
docs2 = [_text.split() for _text in np.array(questions)]
docs3 = [_text.split() for _text in np.array(answers)]

documents = docs1 + docs2 + docs3

<h1>Train Word Embeddings</h1>

Generate word to vec model.

Parameter:
- Vector size : dimensionality of the word vectors.
- Window :  maximum distance between the current and predicted word within a sentence.
- Min Count : ignores all words with total frequency lower than this.
- Workers : put here the number of cores of your cpu to speed up model creation

Build vocabulary and then train and save to file the model pre created with documents.

In [4]:
VECTOR_SIZE = 300
WINDOW_SIZE = 7
EPOCH = 32
MIN_COUNT = 10

In [7]:
word_2_vec_model = gensim.models.word2vec.Word2Vec(vector_size = VECTOR_SIZE, 
                                                   window = WINDOW_SIZE, min_count = MIN_COUNT, workers = 4 )

word_2_vec_model.build_vocab(documents)
words = list(word_2_vec_model.wv.index_to_key)
sizeVocab = len(words)

words
print('Size vocab:', sizeVocab)

word_2_vec_model.train(documents, total_examples = len(documents), epochs = EPOCH)

word_2_vec_model.save('DB/word2vec_trained.bin')

Size vocab: 22736


<h2>Sanity checks on train word embeddings</h2>

In [10]:
word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')


print('Terms most similar to "dataframe"')
print(word_2_vec_model.wv.most_similar("dataframe"))
print('-'*100)

print('Terms most similar to "pandas"')
print(word_2_vec_model.wv.most_similar("pandas"))
print('-'*100)

print('Terms most similar to "indexing"')
print(word_2_vec_model.wv.most_similar("indexing"))

Terms most similar to "dataframe"
[('df', 0.826745331287384), ('column', 0.7716169953346252), ('dataframes', 0.7333992123603821), ('columns', 0.7216950058937073), ('rows', 0.6358416080474854), ('row', 0.6262668371200562), ('series', 0.6258417367935181), ('datframe', 0.6040351986885071), ('multiindex', 0.586420476436615), ('groupby', 0.563845694065094)]
----------------------------------------------------------------------------------------------------
Terms most similar to "pandas"
[('panda', 0.6876720190048218), ('pd', 0.5847082734107971), ('dataframe', 0.5308471322059631), ('pyspark', 0.48259779810905457), ('columns', 0.4754364490509033), ('dataframes', 0.4717135727405548), ('periodindex', 0.4560507833957672), ('groupby', 0.45093220472335815), ('multiindex', 0.4401378035545349), ('df', 0.4367825388908386)]
----------------------------------------------------------------------------------------------------
Terms most similar to "indexing"
[('slicing', 0.7044703364372253), ('slice', 0.